In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext
sc = SparkContext("local", "transformation")

In [3]:
# flatMap, filter이용 공백단어는 제거 
readfile=sc.textFile("input.txt").flatMap(lambda x: x.split(" ")).filter(lambda x: x !="")
# ['A',
#  'STUDY',
#  'IN',
#  'SCARLET.',
#  'PART',
#  'I.',
#  '(Being',
#  'a',,,,]

# readfile.collect()

In [4]:
# 한 글자인 것은 제외, 두 글자이상만 
datas=readfile.filter(lambda x: len(x)>=2)
# ['STUDY',
#  'IN',
#  'SCARLET.',
#  'PART',,,,]

# datas.collect()

In [6]:
# 첫번째 두글자만 추출(slicing)하고 count 위한 key, value 쌍으로
data_tuple = datas.map(lambda x : x[:2]).map(lambda v: (v, 1))
# [('ST', 1),
#  ('IN', 1),
#  ('SC', 1),
#  ('PA', 1),
#  ('I.', 1),
#  ('(B', 1),
#  ('re', 1),
#  ('fr', 1),
#  ('th', 1),
#  ('re', 1),,,]

# data_tuple.collect()

In [7]:
# reduceByKey()  이용 key에 해당하는 value 합
data_reducebykey= data_tuple.reduceByKey(lambda a,b : a+b)
# [('ST', 1),
#  ('IN', 4),
#  ('SC', 2),
#  ('PA', 2),
#  ('I.', 9),
#  ('(B', 1),
#  ('re', 455),,,,]

#data_reducebykey.collect()

# python 방식으로 하면 error
# for idx, v in enumerate(data_tuple):
#     if v in datas:
#         data_tuple[idx] += 1
# print(data_tuple.collect())
# TypeError: 'PipelinedRDD' object is not iterable

In [9]:
# key, value에서 value에 filter # value [1] important
data_filtered = data_reducebykey.filter(lambda v: v[1] <= 32)
# [('ST', 1),
#  ('IN', 4),
#  ('SC', 2),
#  ('PA', 2),
#  ('I.', 9),
#  ('(B', 1),
#  ('JO', 5),
#  ('H.', 2),
#  ('WA', 2),
#  ('M.', 2),
#  ('Ar', 9),,,]

# data_filtered.collect()

In [11]:
#  두글자 count 값이 32보다 작은 값 filter
data_filtered_values=data_filtered.values()
# [1,
#  4,
#  2,
#  2,
#  9,
#  1,,,,]

# data_filtered_values.collect()

In [13]:
sum = 0
for n in data_filtered_values.collect(): 
    sum += n
print(sum)

2214


In [ ]:
# key, value에서 value에 의한 정렬
# sortbyValue() 없음 # sortByKey()  #sortBy 있음
data_sortby=data_reducebykey.sortBy(lambda a: a[1])
#data_sortby.collect()
# [('ST', 1),
#  ('(B', 1),
#  ('Gh', 1),
#  ('Em', 1),
#  ('“p', 1),
#  ('Vo', 1),
#  ('Bi', 1),,,,]

In [ ]:
# mapValue()
# data_mapvalue = data_reducebykey.mapValues( lambda values: map(lambda v:filter(v)<=32 , values )) # Py4JJavaError
# data_mapvalue = data_reducebykey.mapValues( lambda v: v+5 ) # ok

data_mapvalue = data_reducebykey.mapValues(lambda v: v<=32 )  
# [('ST', True),
#  ('IN', True),
#  ('SC', True),
#  ('PA', True),
#  ('I.', True),
#  ('(B', True),
#  ('re', False),
#  ('fr', False),
print(type(data_mapvlaue)) # pyspark.rdd.PipelinedRDD
# data_mapvalue.collect()
#data_mapvalue.collect()


In [ ]:
sc.stop()